In [1]:
pip install BeautifulSoup4

     |████████████████████████████████| 112kB 227kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [30]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [17]:
#Scrape the List of postal codes of Canada

In [31]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [32]:
soup = BeautifulSoup(source, 'xml')

In [5]:
pip install lxml

     |████████████████████████████████| 5.8MB 170kB/s eta 0:00:01     |███████▍                        | 1.3MB 204kB/s eta 0:00:22     |██████████████████████▏         | 4.0MB 1.8MB/s eta 0:00:01|████████████████████████▏       | 4.4MB 1.8MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [34]:
soup = BeautifulSoup(source, 'xml')

In [35]:
table=soup.find('table')

In [36]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [37]:
# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [39]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [40]:
#Data Cleaning: remove rows where Borough is 'Not assigned'

In [45]:
df=df[df['Borough']!='Not assigned']

In [46]:
df[df['Neighborhood']=='Not assigned']
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [47]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [48]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [49]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [50]:
df_merge.drop_duplicates(inplace=True)

In [51]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [52]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park


In [53]:
df_merge.shape

(103, 3)